# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import holoviews as hv
hv.extension('bokeh')
# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.84,88,75,9.52,TF,1731302267
1,1,kamaishi,39.2667,141.8833,16.82,56,8,0.67,JP,1731302268
2,2,avarua,-21.2078,-159.7750,23.03,73,40,4.63,CK,1731302268
3,3,edinburgh of the seven seas,-37.0676,-12.3116,16.20,88,94,15.91,SH,1731302268
4,4,klyuchi,52.2667,79.1667,0.25,98,100,4.94,RU,1731302268


In [3]:
# Check if there are any NaN values
has_nan = city_data_df.isnull().values.any()
print(has_nan)  

True


In [4]:
# Number of NaN values for each column
nan_count = city_data_df.isnull().sum()
print(nan_count)  

City_ID       0
City          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
Country       5
Date          0
dtype: int64


In [5]:
# NaN rows
city_data_df.isnull()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...
571,False,False,False,False,False,False,False,False,False,False
572,False,False,False,False,False,False,False,False,False,False
573,False,False,False,False,False,False,False,False,False,False
574,False,False,False,False,False,False,False,False,False,False


In [6]:
cleaned_df = city_data_df.dropna()
cleaned_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.84,88,75,9.52,TF,1731302267
1,1,kamaishi,39.2667,141.8833,16.82,56,8,0.67,JP,1731302268
2,2,avarua,-21.2078,-159.7750,23.03,73,40,4.63,CK,1731302268
3,3,edinburgh of the seven seas,-37.0676,-12.3116,16.20,88,94,15.91,SH,1731302268
4,4,klyuchi,52.2667,79.1667,0.25,98,100,4.94,RU,1731302268
...,...,...,...,...,...,...,...,...,...,...
571,571,barclayville,4.6797,-8.2339,23.91,93,7,2.39,LR,1731303028
572,572,moose jaw,50.4001,-105.5344,1.14,93,82,1.03,CA,1731303030
573,573,kiruna,67.8557,20.2251,-5.49,93,100,1.03,SE,1731303031
574,574,soure,-0.7167,-48.5233,26.68,79,95,4.08,BR,1731303032


In [7]:
cleaned_df.dtypes

City_ID         int64
City           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
Country        object
Date            int64
dtype: object

In [8]:
cleaned_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 571 entries, 0 to 575
Data columns (total 10 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   City_ID     571 non-null    int64  
 1   City        571 non-null    object 
 2   Lat         571 non-null    float64
 3   Lng         571 non-null    float64
 4   Max Temp    571 non-null    float64
 5   Humidity    571 non-null    int64  
 6   Cloudiness  571 non-null    int64  
 7   Wind Speed  571 non-null    float64
 8   Country     571 non-null    object 
 9   Date        571 non-null    int64  
dtypes: float64(4), int64(4), object(2)
memory usage: 49.1+ KB


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [9]:
# %%capture --no-display

# Configure the map plot
map_plot = cleaned_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [10]:
# Narrow down cities that fit criteria and drop any results with null values
# my ideal weather is 18 - 28 Cencus
ideal_city_data_df = city_data_df[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
clean_ideal_city_data_df = ideal_city_data_df.dropna()


# Display sample data
clean_ideal_city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
75,75,remire-montjoly,4.9167,-52.2667,25.02,100,0,0.00,GF,1731302433
225,225,macapa,0.0389,-51.0664,26.99,89,0,2.57,BR,1731302613
237,237,kolokani,13.5728,-8.0339,21.81,25,0,4.22,ML,1731302628
294,294,mutuali,-14.8706,37.0044,26.84,59,0,2.35,MZ,1731302694
306,306,mermoz boabab,14.7065,-17.4758,24.88,83,0,4.12,SN,1731302707


### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = clean_ideal_city_data_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
75,remire-montjoly,GF,4.9167,-52.2667,100,
225,macapa,BR,0.0389,-51.0664,89,
237,kolokani,ML,13.5728,-8.0339,25,
294,mutuali,MZ,-14.8706,37.0044,59,
306,mermoz boabab,SN,14.7065,-17.4758,83,
323,nguekhokh,SN,14.5128,-17.0050,64,
345,dakhla,EH,23.6848,-15.9580,48,
395,winston,US,28.0317,-82.0148,90,
431,tionk essil,SN,12.7856,-16.5217,41,
440,karrapur,IN,23.9500,78.8500,38,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"   
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitute = hotel_df.loc[index, "Lat"]
    longtitute = hotel_df.loc[index, "Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longtitute},{latitute},{radius}"
    params["bias"] = f"proximity:{longtitute},{latitute}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address_json = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_json["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(5)

Starting hotel search
remire-montjoly - nearest hotel: Complexe Belova
macapa - nearest hotel: Equatorial Hotel
kolokani - nearest hotel: Hôtel kolokani
mutuali - nearest hotel: No hotel found
mermoz boabab - nearest hotel: La maison blanche
nguekhokh - nearest hotel: Dalaal Diam Ecolodge
dakhla - nearest hotel: Hotel Ayouz
winston - nearest hotel: Hilton Garden Inn Lakeland
tionk essil - nearest hotel: Campement Edonia
karrapur - nearest hotel: HOTEl Harsh Residency & Restauraant
kintampo - nearest hotel: No hotel found
rockport - nearest hotel: Holiday Inn Express & Suites
tafi viejo - nearest hotel: Casa Calchaqui


,City,Country,Lat,Lng,Humidity,Hotel Name
75,remire-montjoly,GF,4.9167,-52.2667,100,Complexe Belova
225,macapa,BR,0.0389,-51.0664,89,Equatorial Hotel
237,kolokani,ML,13.5728,-8.0339,25,Hôtel kolokani
294,mutuali,MZ,-14.8706,37.0044,59,No hotel found
306,mermoz boabab,SN,14.7065,-17.4758,83,La maison blanche


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    hover_cols=['Lng','Lat','City','Humidity','Hotel Name','Country']
)
# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)